In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

dataset_path = "<filepath>"
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PetDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

def load_images(folder_path, image_size=IMG_SIZE):
    images, labels, class_names = [], [], sorted(os.listdir(folder_path))
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(folder_path, class_name)
        for file in os.listdir(class_dir):
            img = Image.open(os.path.join(class_dir, file)).convert('RGB').resize((image_size,image_size))
            images.append(np.array(img)/255.0)
            labels.append(label)
    return np.array(images, dtype=np.float32), np.array(labels), class_names

X, y, class_names = load_images(dataset_path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = PetDataset(X_train, y_train, transform=transform)
test_dataset = PetDataset(X_test, y_test, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*32*32,128), nn.ReLU(),
            nn.Linear(128,len(class_names))
        )
    def forward(self,x):
        return self.fc(self.conv(x))

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses, val_losses, train_acc, val_acc = [], [], [], []

for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0,0,0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs,1)
        correct += (preds==labels).sum().item()
        total += labels.size(0)
    train_losses.append(running_loss/len(train_loader))
    train_acc.append(correct/total)

    model.eval()
    val_loss, correct, total = 0,0,0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs,1)
            correct += (preds==labels).sum().item()
            total += labels.size(0)
    val_losses.append(val_loss/len(test_loader))
    val_acc.append(correct/total)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Acc: {train_acc[-1]:.3f}, Val Acc: {val_acc[-1]:.3f}")

y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = torch.argmax(outputs,1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(train_acc,label='Train Accuracy')
plt.plot(val_acc,label='Val Accuracy')
plt.title('Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(train_losses,label='Train Loss')
plt.plot(val_losses,label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()